In [ ]:
import torch
import torch.nn as nn
import numpy as np
import cv2
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import os
import pandas as pd
from skimage import io
from tqdm.notebook import tqdm
import gc

In [ ]:
torch.cuda.empty_cache()
# del variables
# gc.collect()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

learning_rate = 3e-4
batch_size = 4
num_epochs = 70
# in_channels = 4

cuda


In [ ]:
class LogCoshLoss(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, y_t, y_prime_t):
        ey_t = y_t - y_prime_t
        return torch.mean(torch.log(torch.cosh(ey_t + 1e-12)))

# class Identity(nn.Module):
#   def __init__(self):
#     super(Identity,self).__init__()

#   def forward(self, x):
#     return x

class PrintLayer(nn.Module):
    def __init__(self):
        super(PrintLayer, self).__init__()
    
    def forward(self, x):
        # Do your print / debug stuff here
        print(x)
        return x

In [ ]:
class Rotated_CCS1_CCS2(Dataset):
  def __init__(self, csv_file, root_dir, transform=None):
    self.annotations = pd.read_csv(csv_file)
    self.root_dir = root_dir
    self.transform = transform
  
  def __len__(self):
    return len(self.annotations)

  def __getitem__(self,index):
    img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0]+'.png')
    image = cv2.imread(img_path)
    y_label = torch.tensor(float(self.annotations.iloc[index, 1]))

    if self.transform:
      image = self.transform(image)
      # print(torch.isnan(data).any())
    return (image, y_label)

In [ ]:
model = torchvision.models.vgg16()
# model.classifier[6] = Linear(in_features = 4096, out_features = 1024, bias=True)
# model.classifier[7] = Softmax(in_features = 1024, out_features = 1)
# for param in model.parameters():
#   param.requires_grad = False
# # model.features[0] = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
# model.classifier = nn.Sequential(nn.Linear(in_features = 25088, out_features = 4096, bias=True),
#                                  nn.ReLU(inplace=True),
#                                  nn.Dropout(p=0.5, inplace=False),
#                                  nn.Linear(in_features = 4096, out_features = 4096, bias=True),
#                                  nn.ReLU(inplace=True),
#                                  nn.Dropout(p=0.5, inplace=False),
#                                  nn.Linear(in_features = 4096, out_features = 1024, bias=True),
#                                  nn.ReLU(inplace=True),
#                                  nn.Dropout(p=0.5, inplace=False),
#                                  nn.Linear(in_features = 1024, out_features = 1, bias=True),
#                                  nn.Sigmoid())

print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
model.classifier[6] = nn.Linear(in_features=4096, out_features=1, bias=True)

model.classifier = nn.Sequential(
    nn.Linear(in_features=25088, out_features=4096, bias=True),
    PrintLayer(),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=4096, out_features=4096, bias=True),
    PrintLayer(),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=4096, out_features=1, bias=True)
)

In [ ]:
# model.classifier = model.classifier[:4] 

In [ ]:
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
# model.avgpool = Identity()
# model.classifier = torch.nn.Sequential(
#     nn.Linear(in_features=512, out_features=128, bias=True),
#     nn.ReLU(inplace=True),
#     nn.Dropout(p=0.5, inplace = True),
#     nn.Linear(in_features=128, out_features=1, bias=True)
#     # torch.nn.Sigmoid()
# )

In [ ]:
# model.to(device)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
dataset = Rotated_CCS1_CCS2(csv_file='/content/drive/MyDrive/data/rotatedportDegrees.csv', root_dir = '/content/drive/MyDrive/data', transform = transforms.Compose(
    [transforms.ToTensor(),
    torchvision.models.VGG16_Weights.IMAGENET1K_FEATURES.transforms()]))
print(len(dataset))
train_set, test_set = torch.utils.data.random_split(dataset, [0.75, 0.25])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True, pin_memory=True)
test_loader = DataLoader(dataset =test_set, batch_size=batch_size, shuffle=False, pin_memory=True)

100


In [ ]:
criterion = nn.MSELoss()
# criterion = LogCoshLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
i = 1
sum = 0
def save_checkpoint(state, filename = "/content/drive/MyDrive/data/saved_model1.pth.tar"):
  print("..Saving Model..")
  torch.save(state, filename)

def load_checkpoint(checkpoint):
    print("..Loading Checkpoint..")
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])

In [ ]:
for epoch in range (num_epochs):
  sum = 0
  for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
    data=data.to(device=device, non_blocking=True)
    targets=targets.to(device=device, non_blocking=True)
    targets = targets.reshape(-1, 1)
    scores = model(data)
    print(targets.shape)
    print(scores.shape)
    print(scores, targets)
    loss = torch.sqrt(criterion(scores, targets))
    sum += loss

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print(f'loss = {float(sum)/float(len(train_loader)) * 100:.2f}%')
  

  0%|          | 0/19 [00:00<?, ?it/s]

tensor([[-0.0346, -0.0162, -0.0448,  ..., -0.0321, -0.0132, -0.0103],
        [-0.0275, -0.0145, -0.0511,  ..., -0.0427, -0.0060, -0.0198],
        [-0.0276, -0.0123, -0.0340,  ..., -0.0022, -0.0122, -0.0194],
        [-0.0297, -0.0141, -0.0132,  ...,  0.0031, -0.0103, -0.0209]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-0.0066, -0.0251, -0.0040,  ..., -0.0117,  0.0077, -0.0190],
        [-0.0257, -0.0317, -0.0237,  ..., -0.0173,  0.0147, -0.0272],
        [-0.0349, -0.0181,  0.0244,  ..., -0.0480, -0.0245, -0.0261],
        [-0.0127, -0.0214,  0.0055,  ..., -0.0268, -0.0261, -0.0240]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([4, 1])
torch.Size([4, 1])
tensor([[0.4421],
        [0.3554],
        [0.9541],
        [0.7580]], device='cuda:0', grad_fn=<AddmmBackward0>) tensor([[0.4497],
        [0.3833],
        [0.9444],
        [0.6844]], device='cuda:0')
tensor([[-0.0298, -0.0111, -0.0445,  ..., -0.0351, -0.0103, -0.0152],
        [-0.0339, -0.011

  0%|          | 0/19 [00:00<?, ?it/s]

tensor([[-0.0278, -0.0103, -0.0376,  ..., -0.0277, -0.0070, -0.0136],
        [-0.0287, -0.0130, -0.0438,  ..., -0.0240, -0.0093, -0.0159],
        [-0.0304, -0.0152, -0.0345,  ..., -0.0351, -0.0080, -0.0106],
        [-0.0334, -0.0110, -0.0379,  ..., -0.0309, -0.0079, -0.0145]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-0.0096, -0.0231,  0.0090,  ..., -0.0221, -0.0044, -0.0201],
        [-0.0094, -0.0247,  0.0007,  ..., -0.0137,  0.0095, -0.0193],
        [-0.0079, -0.0212,  0.0050,  ..., -0.0118,  0.0050, -0.0217],
        [-0.0089, -0.0209,  0.0010,  ..., -0.0148,  0.0136, -0.0206]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([4, 1])
torch.Size([4, 1])
tensor([[0.6061],
        [0.4850],
        [0.5275],
        [0.4614]], device='cuda:0', grad_fn=<AddmmBackward0>) tensor([[0.6722],
        [0.4683],
        [0.5381],
        [0.4489]], device='cuda:0')
tensor([[-0.0248, -0.0102, -0.0177,  ..., -0.0220, -0.0083, -0.0112],
        [-0.0243, -0.009

  0%|          | 0/19 [00:00<?, ?it/s]

...

tensor([[-0.0215, -0.0103, -0.0139,  ..., -0.0176, -0.0139, -0.0117],
        [-0.0104, -0.0091, -0.0254,  ..., -0.0186, -0.0149, -0.0172],
        [-0.0189, -0.0090, -0.0175,  ..., -0.0194, -0.0122, -0.0110],
        [-0.0092, -0.0089, -0.0275,  ..., -0.0291, -0.0151, -0.0111]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-0.0099, -0.0210,  0.0098,  ..., -0.0202, -0.0184, -0.0201],
        [-0.0154, -0.0248,  0.0133,  ..., -0.0220, -0.0238, -0.0244],
        [-0.0103, -0.0205,  0.0086,  ..., -0.0156,  0.0044, -0.0209],
        [-0.0075, -0.0197,  0.0083,  ..., -0.0150,  0.0058, -0.0214]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([4, 1])
torch.Size([4, 1])
tensor([[0.6309],
        [0.7493],
        [0.5159],
        [0.5393]], device='cuda:0', grad_fn=<AddmmBackward0>) tensor([[0.6844],
        [0.7500],
        [0.5103],
        [0.5278]], device='cuda:0')
tensor([[-0.0207, -0.0121, -0.0328,  ..., -0.0334, -0.0130, -0.0149],
        [-0.0203, -0.012

  0%|          | 0/19 [00:00<?, ?it/s]

tensor([[-0.0196, -0.0106, -0.0301,  ..., -0.0266, -0.0127, -0.0093],
        [-0.0230, -0.0105, -0.0139,  ..., -0.0100, -0.0125, -0.0089],
        [-0.0085, -0.0112, -0.0382,  ..., -0.0537, -0.0174, -0.0158],
        [-0.0164, -0.0091, -0.0191,  ..., -0.0108, -0.0119, -0.0110]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-0.0157, -0.0206,  0.0055,  ..., -0.0152,  0.0035, -0.0189],
        [-0.0135, -0.0218,  0.0126,  ..., -0.0195, -0.0127, -0.0216],
        [-0.0263, -0.0233, -0.0035,  ..., -0.0245,  0.0098, -0.0272],
        [-0.0160, -0.0298,  0.0211,  ..., -0.0297, -0.0327, -0.0240]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([4, 1])
torch.Size([4, 1])
tensor([[0.4588],
        [0.6703],
        [0.3603],
        [0.9959]], device='cuda:0', grad_fn=<AddmmBackward0>) tensor([[0.4683],
        [0.6669],
        [0.4028],
        [0.9861]], device='cuda:0')
tensor([[-0.0171, -0.0073, -0.0328,  ..., -0.0252, -0.0122, -0.0135],
        [-0.0161, -0.009

  0%|          | 0/19 [00:00<?, ?it/s]

tensor([[-0.0247, -0.0098, -0.0317,  ..., -0.0327, -0.0107, -0.0100],
        [-0.0149, -0.0093, -0.0183,  ..., -0.0056, -0.0124, -0.0117],
        [-0.0201, -0.0089, -0.0243,  ..., -0.0109, -0.0093, -0.0106],
        [-0.0184, -0.0085, -0.0208,  ..., -0.0053, -0.0130, -0.0105]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-0.0238, -0.0232,  0.0045,  ..., -0.0174,  0.0017, -0.0298],
        [-0.0099, -0.0241,  0.0018,  ..., -0.0201, -0.0182, -0.0222],
        [-0.0173, -0.0236,  0.0102,  ..., -0.0280, -0.0140, -0.0276],
        [-0.0284, -0.0268,  0.0330,  ..., -0.0360, -0.0297, -0.0270]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([4, 1])
torch.Size([4, 1])
tensor([[0.3986],
        [0.7734],
        [0.7932],
        [0.9222]], device='cuda:0', grad_fn=<AddmmBackward0>) tensor([[0.3833],
        [0.6806],
        [0.7611],
        [0.9444]], device='cuda:0')
tensor([[-0.0228, -0.0107, -0.0278,  ..., -0.0254, -0.0121, -0.0089],
        [-0.0314, -0.011

In [ ]:

def check_accuracy(loader, model, desAcc):
    # if loader.dataset.training:
    #     print('Checking accuracy on training data')
    # else:
    #     print('Checking accuracy on test data')
    index = np.zeros((25,))
    num_correct = 0
    num_samples = 0
    model.eval()
    # i=1
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            # x = x.reshape(x.shape[0], -1)
            y = y.reshape(y.shape[0], -1)
            scores = model(x)
            
            # _, predictions = scores.max(1)  # dimension of predictions
            # predictions = torch.argmax(scores, dim=1)
            # new_y = torch.argmax(y, dim=2)
            # _, predictions = scores.max(1)
            # if i<22:
            #   predictions = scores.reshape(4)
            # else:
            #   predictions = scores.reshape(1)
            # i+=1
            predictions = scores.reshape(scores.shape[0], -1)
            # print(predictions, "--", y)
            # print(y)
            # print(desAcc*y)
            num_correct += (torch.abs(predictions - y) < desAcc*y).sum()
            num_samples += predictions.size(0)

        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct) / float(num_samples) * 100:.2f} %')
    model.train()

In [ ]:
# check_accuracy(train_loader, model, desAcc = 0.1)
check_accuracy(test_loader, model, desAcc = 0.1)

tensor([[-0.0210, -0.0101, -0.0180,  ..., -0.0275, -0.0140, -0.0141],
        [-0.0217, -0.0093, -0.0165,  ..., -0.0135, -0.0125, -0.0104],
        [-0.0309, -0.0110, -0.0289,  ..., -0.0591, -0.0174, -0.0068],
        [-0.0226, -0.0104, -0.0346,  ..., -0.0443, -0.0169, -0.0066]],
       device='cuda:0')
tensor([[-0.0096, -0.0209,  0.0093,  ..., -0.0146,  0.0062, -0.0215],
        [-0.0163, -0.0219,  0.0131,  ..., -0.0241, -0.0161, -0.0277],
        [-0.0096, -0.0207,  0.0085,  ..., -0.0140,  0.0069, -0.0223],
        [-0.0060, -0.0207,  0.0087,  ..., -0.0136,  0.0031, -0.0225]],
       device='cuda:0')
tensor([[-0.0124, -0.0101, -0.0247,  ..., -0.0220, -0.0142, -0.0095],
        [-0.0099, -0.0084, -0.0396,  ..., -0.0543, -0.0209, -0.0067],
        [-0.0320, -0.0107, -0.0256,  ..., -0.0456, -0.0174, -0.0199],
        [-0.0101, -0.0115, -0.0289,  ..., -0.0367, -0.0151, -0.0170]],
       device='cuda:0')
tensor([[-0.0261, -0.0293,  0.0017,  ..., -0.0217,  0.0095, -0.0267],
        [-0.037

In [ ]:
checkpoint = {'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict()}
save_checkpoint(checkpoint)

..Saving Model..


In [ ]:
for params in model.parameters():
  print(params)

In [ ]:
load_checkpoint(torch.load("/content/drive/MyDrive/data/saved_model.pth.tar"))

In [ ]:
index = np.zeros((28000,))